<a href="https://colab.research.google.com/github/modhudeb/determine-Pneumonia-Xray/blob/main/3_Xray_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
from skimage import io, transform

In [21]:
import tensorflow as tf
from tensorflow.keras import models, layers

# Loading

In [4]:
df = pd.read_csv("/content/drive/MyDrive/data/files/xrayImg.csv", index_col='Num')

In [5]:
ndf = pd.read_csv("/content/drive/MyDrive/data/files/xray_test.csv", index_col='Num')

# Converting and splitting

In [6]:
features = df.drop(columns=['label'])
targets = df['label']

In [7]:
nfeatures = ndf.drop(columns=['label'])
ntargets = ndf['label']

In [8]:
class df_to_npImg :
  """This class converts your dataframe to numpy array and reshapes it with
    given height and width.
    N:B: use it only if your images were flattened before adding to pandas dataframe
  """
  def __init__(self) -> None:
    pass
  
  def apply(dframe:pd.DataFrame, height:int, width:int) -> np.array :
    images = dframe.to_numpy()
    arr = list(map(lambda z : z.reshape(height, width), images))
    arr = np.asarray(arr)
    return arr

In [9]:
xtrain = df_to_npImg.apply(features, 150,150)
xtest = df_to_npImg.apply(nfeatures, 150,150)

ytrain = targets
ytest = ntargets

In [10]:
xtrain.shape

(5155, 150, 150)

In [11]:
xtest.shape

(624, 150, 150)

In [59]:
train_dataset = tf.data.Dataset.from_tensor_slices((xtrain, ytrain))
test_dataset = tf.data.Dataset.from_tensor_slices((xtest, ytest))

In [60]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# CNN model

In [29]:
from tensorflow import keras

In [71]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3,3), activation = 'relu', input_shape = (150,150, 1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(filters=64, kernel_size=(3,3), activation = 'selu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(filters=16, kernel_size=(3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),

    layers.Dense(32, activation = 'relu'),
    layers.Dense(1, activation = 'sigmoid')
  ])



In [72]:
with tf.device('/device:GPU:0'):
  cnn.compile(optimizer=keras.optimizers.RMSprop(),
            loss = keras.losses.BinaryCrossentropy(),
            metrics = ['accuracy']
            )
  cnn.fit(train_dataset, epochs=20, validation_data=test_dataset)

Epoch 1/20
81/81 [==============================] - 5s 46ms/step - loss: 0.1828 - accuracy: 0.9571 - val_loss: 7.4700 - val_accuracy: 0.6250
Epoch 2/20
81/81 [==============================] - 4s 44ms/step - loss: 0.3396 - accuracy: 0.9606 - val_loss: 6.5316 - val_accuracy: 0.6250
Epoch 3/20
81/81 [==============================] - 4s 44ms/step - loss: 0.2885 - accuracy: 0.9618 - val_loss: 5.3760 - val_accuracy: 0.6250
Epoch 4/20
81/81 [==============================] - 4s 43ms/step - loss: 0.2421 - accuracy: 0.9639 - val_loss: 5.7224 - val_accuracy: 0.6250
Epoch 5/20
81/81 [==============================] - 4s 43ms/step - loss: 0.2332 - accuracy: 0.9746 - val_loss: 7.1924 - val_accuracy: 0.6250
Epoch 6/20
81/81 [==============================] - 4s 44ms/step - loss: 0.2958 - accuracy: 0.9738 - val_loss: 5.7071 - val_accuracy: 0.6250
Epoch 7/20
81/81 [==============================] - 4s 43ms/step - loss: 0.2327 - accuracy: 0.9746 - val_loss: 4.9340 - val_accuracy: 0.6250
Epoch 8/20
81